[View in Colaboratory](https://colab.research.google.com/github/vigneshrao98/Colab-Keras-CNN2/blob/master/Keras_DeepCNN.ipynb)

## Fairly complex CNN on CIFAR-10 dataset

Following is the implementation of CNN with 6 Conv layers and 3 MaxPool layers followed by a Fully conected layer.


It is to be noted that a complex CNN on a small dataset like CIFAR-10 usually leads to overfitting so a good amout of regularization is done to achieve a impressive 89% accuracy with only 50,000 training images compared to the 78% accuracy achieved by the shallow network.

Details of the implementation in the README file.

In [0]:
from google.colab import files


In [3]:
files.upload()

Saving distutils3.py to distutils3.py
Saving Parameters.py to Parameters.py


{'Parameters.py': b'#regularization\r\nweight_decay=0.0001\r\nepochs=140\r\n#for rmsprop\r\nalpha=0.001  \r\nalpha_decay=0.000001\r\nbatch_size=64',
 'distutils3.py': b'#from sklearn.datasets import fetch_mldata\r\nfrom sklearn.preprocessing import OneHotEncoder\r\nfrom matplotlib import pyplot\r\nfrom scipy.misc import toimage\r\n#from sklearn.model_selection import train_test_split\r\nfrom urllib.request import urlretrieve\r\nfrom keras.preprocessing.image import ImageDataGenerator\r\nimport numpy as np\r\nimport os\r\nimport tarfile\r\nimport pickle\r\n\r\ndef lrScheduler(epoch):\r\n    lrate = 0.001\r\n    if epoch > 75:\r\n        lrate = 0.0005\r\n    elif epoch > 100:\r\n        lrate = 0.0003        \r\n    return lrate\r\n\r\ndef displayImg(X):\r\n    pyplot.figure(1)\r\n    k = 0\r\n    for i in range(0,4):\r\n        for j in range(0,4):\r\n            pyplot.subplot2grid((4,4),(i,j))\r\n            pyplot.imshow(toimage(X[k]))\r\n            k = k+1\r\n    # show the plot\r

In [18]:
files.download('model2.h5')

In [0]:
files.os.listdir()

*Import Libraries and Utility functions*


In [12]:
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.optimizers import rmsprop
from keras.callbacks import LearningRateScheduler
from distutils3 import *
from Parameters import *

In [13]:
x_train, x_test, y_train, y_test = getCifarDataset()
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)


Preparing train and test sets
Preparing test set...
Done.
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)
float32 float32 int32 int32


*Defining the Model*

In [14]:
def createModel():
  model = Sequential()
  model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.3))

  model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))
  return model


*Image Augmentation*

In [15]:
augmentedX_train=augmentImgs(x_train)

In [16]:
mod = createModel()

In [17]:
def init_model(m):
    m.compile(loss='categorical_crossentropy', 
                  optimizer=rmsprop(lr=alpha,decay=alpha_decay), 
                  metrics=['accuracy'])
    return m

In [18]:
model = init_model(mod)

In [0]:
model.fit_generator(augmentedX_train.flow(x_train, y_train, batch_size=128),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lrScheduler)])

In [15]:
model_json = model.to_json()

with open('model.json', 'w') as json_file:
    json_file.write(model_json)
    
model.save_weights('model2.h5')    

In [16]:
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 2s 185us/step

Test result: 80.370 loss: 0.716
